In [ ]:
# Block 0: Documentation

print('Program to process and visualize ABI AOD data, AMS Short Course, March 18, 2021\n')
print('Version 1.0, Februrary 8, 2020\n')
print('Written by Dr. Amy Huff (IMSG at NOAA/NESDIS/STAR) and Ryan Theurer (GVT LLC at NOAA/NESDIS/STAR)\n')
print('For questions contact Dr. Huff: amy.huff@noaa.gov\n')
print('This program processes ABI aerosol optical depth (AOD) using data quality flags (DQF) and plots processed AOD on a map to create a professional-looking figure.\n')
print('Block 1 imports modules and libraries, and blocks 2-9 are functions that require no input from the user; there is no visible output from these blocks. In block 10, the user enters settings and obtains output.')

In [ ]:
# Block 1: Import modules and libraries

# Library to perform array operations
import numpy as np

# Main plotting libraries
import matplotlib as mpl
from matplotlib import pyplot as plt
import matplotlib.patheffects as PathEffects

# Libraries to create maps
from cartopy import crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import cartopy.io.shapereader as shpreader

# Module to access files in the directory
import os

# Library to read netCDF files
from netCDF4 import Dataset

# Module for manipulating dates and times
import datetime

# Module to collect lists of files from folders
import glob

# Library controlling warning messages
# Cartopy can trigger many irrelevant warning messages via matplotlib
# Recommend ignorning these distracting warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Block 2: Algorithm to calculate latitude and longitude in degrees from GOES fixed grid projection data

def degrees(file_id):
    # Ignore numpy error for sqrt of negative number ('x'); occurs for GOES-16 ABI CONUS sector data
    np.seterr(invalid='ignore')
    
    # Read in GOES Imager Projection data
    lat_rad_1d = file_id.variables['x'][:]
    lon_rad_1d = file_id.variables['y'][:]
    projection_info = file_id.variables['goes_imager_projection']
    lon_origin = projection_info.longitude_of_projection_origin
    H = projection_info.perspective_point_height+projection_info.semi_major_axis
    r_eq = projection_info.semi_major_axis
    r_pol = projection_info.semi_minor_axis
    
    # Create meshgrid filled with radian angles
    lat_rad,lon_rad = np.meshgrid(lat_rad_1d,lon_rad_1d)
    
    # lat/lon calculus routine from satellite radian angle vectors
    lambda_0 = (lon_origin*np.pi)/180.0
    
    a_var = np.power(np.sin(lat_rad),2.0) + (np.power(np.cos(lat_rad),2.0)*(np.power(np.cos(lon_rad),2.0)+(((r_eq*r_eq)/(r_pol*r_pol))*np.power(np.sin(lon_rad),2.0))))
    b_var = -2.0*H*np.cos(lat_rad)*np.cos(lon_rad)
    c_var = (H**2.0)-(r_eq**2.0)
    
    r_s = (-1.0*b_var - np.sqrt((b_var**2)-(4.0*a_var*c_var)))/(2.0*a_var)
    
    s_x = r_s*np.cos(lat_rad)*np.cos(lon_rad)
    s_y = - r_s*np.sin(lat_rad)
    s_z = r_s*np.cos(lat_rad)*np.sin(lon_rad)
    
    lat = (180.0/np.pi)*(np.arctan(((r_eq*r_eq)/(r_pol*r_pol))*((s_z/np.sqrt(((H-s_x)*(H-s_x))+(s_y*s_y))))))
    lon = (lambda_0 - np.arctan(s_y/(H-s_x)))*(180.0/np.pi)
    
    return lat, lon

In [ ]:
# Block 3: Select and process AOD data using DQF
# "quality" is a global variable set in final block

def aod_data(file_id, quality):
    # Read in AOD data
    aod_data = file_id.variables['AOD'][:,:]

    # Select quality of AOD data by masking pixels we don't want to display using DQF variable
    # high quality: DQF = 0, medium quality: DQF = 1, low quality: DQF = 2, not retrieved (NR): DQF = 3
    # Use high + medium qualities ("top 2 qualities") for operational applications (e.g., mask low quality and NR pixels)
    dqf = file_id.variables['DQF'][:,:]
    if quality == 'high':
        quality_mask = (dqf != 0)
    if quality == 'top2':
        quality_mask = (dqf > 1)
    if quality == 'all':
        quality_mask = (dqf > 2)
    aod = np.ma.masked_where(quality_mask, aod_data) 
    
    return aod

In [ ]:
# Block 4: Create AOD color bar, indepenent of plotted data, auto-positioning
# cbar_ax are dummy variables
# Location/dimensions of colorbar set by .set_position (x0, y0, width, height) to scale automatically with figure
# "bar" (color map) and "max_color" (color for AOD > 1) are global variables set in final block

def aod_colorbar(fig, ax, bar, max_color):
    last_axes = plt.gca()
    cbar_ax = fig.add_axes([0, 0, 0, 0])
    plt.draw()
    posn = ax.get_position()
    cbar_ax.set_position([0.35, posn.y0 - 0.06, 0.3, 0.02])
    norm = mpl.colors.Normalize(vmin=0, vmax=1)
    color_map = plt.get_cmap(bar)
    color_map.set_over(max_color)
    cb = mpl.colorbar.ColorbarBase(cbar_ax, cmap=color_map, norm=norm, orientation='horizontal', ticks=[0, 0.25, 0.5, 0.75, 1], extend='max')
    cb.set_label(label='Aerosol Optical Depth', size=10, weight='bold')
    cb.ax.set_xticklabels(['0', '0.25', '0.50', '0.75', '1.0'])
    cb.ax.tick_params(labelsize=10)
    plt.sca(last_axes)

In [ ]:
# Block 5: Set range for plotting AOD data (min = 0, max = 1)
# "contour_interval" is a global variable set in final block

def aod_data_range(contour_interval):    
    aod_data_range = np.arange(0, 1.1, contour_interval)
    
    return aod_data_range

In [ ]:
# Block 6: Format map with lat/lon ticks, coastlines/borders, and set map domain
# "lon_ticks", "lat_ticks", and "domain" are global variables set in final block

def map_settings(ax, lon_ticks, lat_ticks, domain):
    # Set up and label the lat/lon grid
    ax.xaxis.set_major_formatter(LongitudeFormatter())
    ax.yaxis.set_major_formatter(LatitudeFormatter())
    ax.set_xticks(lon_ticks, crs=ccrs.PlateCarree())
    ax.set_yticks(lat_ticks, crs=ccrs.PlateCarree())
    
    # Set lat/lon ticks
    ax.tick_params(length=5, direction='out', labelsize=10, pad=5)
   
    #Draw coastlines/borders using Cartopy; zorder sets drawing order for layers
    ax.coastlines(resolution='50m', zorder=3)
    ax.add_feature(cfeature.NaturalEarthFeature(category='physical', name='ocean', scale='50m'), facecolor='lightgrey')
    ax.add_feature(cfeature.NaturalEarthFeature(category='physical', name='land', scale='50m'), facecolor='grey')
    ax.add_feature(cfeature.NaturalEarthFeature(category='physical', name='lakes', scale='50m'), facecolor='lightgrey', edgecolor='black', zorder=1.5)
    ax.add_feature(cfeature.NaturalEarthFeature(category='physical', name='lakes', scale='50m'), facecolor='none', edgecolor='black', zorder=2.5)
    ax.add_feature(cfeature.NaturalEarthFeature(category='cultural', name='admin_0_countries', scale='50m'), facecolor='none', lw=0.5, edgecolor='black', zorder=2.5)
    ax.add_feature(cfeature.NaturalEarthFeature(category='cultural', name='admin_1_states_provinces', scale='50m'), facecolor='none', lw=0.5, edgecolor='black', zorder=2.5)
    
    # Set lat/lon extents for map
    ax.set_extent(domain, crs=ccrs.PlateCarree())

In [ ]:
# Block 7: Add city names with filled circle marker to map
# "city_res", "city_names", "city_marker", and "city_text" are global variables set in final block

def add_cities(ax, city_res, city_names, city_marker, city_text):
    shpfilename = shpreader.natural_earth(city_res, category='cultural', name='populated_places')
    reader = shpreader.Reader(shpfilename)
    places = reader.records()
    for place in places:
        if place.attributes['NAME'] in city_names:
            x = place.geometry.centroid.x        
            y = place.geometry.centroid.y
            ax.plot(x, y, 'wo', ms=city_marker, mec='black', zorder=5, transform=ccrs.PlateCarree())
            ax.text(x + 0.1, y + 0.15, place.attributes['NAME'], color='white', weight='bold', size=city_text, ha='center', va='center', path_effects=[PathEffects.withStroke(linewidth=1, foreground='black')], zorder=5, transform=ccrs.PlateCarree())
        else:
            pass

In [ ]:
# Block 8: Create plot title from ABI data file name
# "quality" is a global variable set in final block

def abi_plot_title(fname, quality):
    # Pull Julian date from file name, convert to Gregorian date, and reformat
    julian = datetime.datetime.strptime(fname[-49:-42], '%Y%j').date()
    date = julian.strftime('%d %b %Y')
    
    # Set AOD quality level
    if quality == 'high':
        quality_title = '(High Quality)'
    if quality == 'top2':
        quality_title = '(High and Medium Quality)'
    if quality == 'all':
        quality_title = '(High, Medium, and Low Quality)'

    # Create plot title
    abi_title = 'GOES-'+ fname[-53:-51] + '/ABI\n' + 'Aerosol Optical Depth ' + quality_title + '\n' + fname[-42:-40] + ':' + fname[-40:-38] + ' UTC, ' + date
        
    return abi_title

In [ ]:
# Block 9: Plot ABI AOD data on Plate Carree map projection
# "save_path" and file_res" are global variables set in final block

def plot_abi_aod(file_id, save_path, file_res):

    # Set up figure and map projection: PlateCarree(central_longitude)
    # Plate Carree: equidistant cylindrical projection w/equator as the standard parallel; default central_longitude = 0
    fig = plt.figure(figsize=(8, 10))
    ax = plt.axes(projection=ccrs.PlateCarree())

    # Select and process ABI AOD data
    aod = aod_data(file_id, quality)

    # Read in ABI latitutude and longitude values in degrees
    lat, lon = degrees(file_id)

    # Format map
    map_settings(ax, lon_ticks, lat_ticks, domain)

    # Add title
    plt.title(abi_plot_title(fname, quality), pad=10, ma='center', size=12, weight='bold')
    
    # Add AOD color bar
    aod_colorbar(fig, ax, bar, max_color)
    
    # Add city name/marker
    if len(city_names) > 0:
        add_cities(ax, city_res, city_names, city_marker, city_text)
    else:
        pass

    # Plotting settings for AOD data
    data_range = aod_data_range(contour_interval)
    color_map = plt.get_cmap(bar)
    color_map.set_over(max_color)

    # Create filled contour plot of AOD data
    ax.contourf(lon, lat, aod, data_range, cmap=color_map, extend='both', zorder=2.5, transform=ccrs.PlateCarree())

    # Show figure
    plt.show()

    # Save figure as a .png file with "save_name" to specified directory
    date = datetime.datetime.strptime(fname[-49:-42], '%Y%j').date().strftime('%Y%m%d')
    save_name = 'G' + fname[-53:-51] + '_ABI_AOD_' + quality + '_' + date + '_' + fname[-42:-38]
    fig.savefig(save_path + save_name, bbox_inches='tight', dpi=file_res)
    
    # Close file
    file_id.close()
    
    # Erase plot so we can build the next one
    plt.close()

In [ ]:
# Block 10: Enter user settings and generate figure(s)
# Recommend setting "figures" = "single" first to confirm data, plot, mapping, label settings
# Once all settings are satisfactory, then set "figures" = "multiple"

# File settings
file_path = os.getcwd() + '/'  # Directory where ABI data files are located
save_path = os.getcwd() + '/'  # Directory where figures will be saved
figures = 'single'  # Number of figures to create: 'single' (one data file) or 'multiple' (multiple data files)
file_name = 'OR_ABI-L2-AODC-M6_G17_s20203382001177_e20203382003550_c20203382005376.nc'  # For plotting a single file

# Data settings
quality = 'top2'  # AOD data quality: 'high', 'top2', or 'all'
contour_interval = 0.05  # AOD contour interval: 0.1 = runs faster/coarser resolution, 0.01 = runs slower/higher resolution

# Plot settings
bar = 'rainbow'  # Color map for colorbar and plot
max_color = 'darkred'  # Color for AOD data > 1
file_res = 150  # DPI setting for image resolution

# Mapping settings
# Use 180 degrees for longitude (i.e, 100 degrees W = -100)
domain = [-121, -116, 32, 35]  # Boundaries of map: [western lon, eastern lon, southern lat, northern lat] 
lon_ticks = [-120, -118, -116]
lat_ticks = [32, 33, 34, 35]

# City label settings
city_names = ['Los Angeles']  # Note: Spelling must match shapefile (including any accents)
city_res = '110m'  # Resolution of shapefile: '110m', '50m', '10m'
city_text = 12  # Font size for city label
city_marker = 8  # Size for filled circle marker indicating city location

##########################################################################################################################

if __name__ == "__main__":

    if figures == 'single':
        # Open single data file and set file name
        fname = file_path + file_name
        file_id = Dataset(fname)
        plot_abi_aod(file_id, save_path, file_res)
        print('Done!')

    if figures == 'multiple':
        # Collect all of the .nc files in specified directory
        file_list = sorted(glob.glob(file_path + '*.nc')) 
        # Loop through data files, making/saving a figure for each data file
        for fname in file_list:
            file_id = Dataset(fname)
            plot_abi_aod(file_id, save_path, file_res)
        print('Done!')